The CrosswordFiller needs two things as input: an empty crossword (which specifies the locations of black fields) and a vocabulary.

The vocabulary is a list of words that can be used in the crossword. For the final product, of course, the words in the crossword need to be connect to clues. This can be done post hoc, or the CrosswordFiller could be adapted to import a dictionary of words and clues, and export clues as part of the output.

As a bit of terminology:

A <b>crossword</b> is the actual puzzle, meaning the grid structure. The <b>solution</b> to a crossword is the grid completely filled out.

A crossword consists of <b>fields</b>. A field has a single <b>letter</b>. This is usually a single character in the strict sense, but Dutch crosswords count "IJ"  as a single letter. Fields that have no corresponding letter are called <b>black fields</b>.

Fields in crosswords have coordinates (X,Y). X coordinates are counted left to right, starting at 0. Y coordinates are counted top to bottom, starting at 0.

A <b>sequence</b> is a group of fields on a horizontal or vertical line that correspond to a single word in the solution. Each sequence has a corresponding clue.

A <b>word</b> is an ordered set of letters and an element of the vocabulary. A word can be filled into a sequence.

## Vocabulary import

In [166]:
#use the NLTK words corpus as a filler vocabulary

import nltk
#nltk.download()

nltkwords = nltk.corpus.words.words()
unfilteredvocab = [w.lower() for w in nltkwords]

vocab = []
for w in unfilteredvocab:
    if not '-' in w:
        vocab.append(w)
        

In [169]:
#organise the vocab by length

vocabdict = dict()
for w in vocab:
    l = len(w)
    if l in vocabdict.keys():
        vocabdict[l].add(w)
    else:
        vocabdict[l] = {w}
        
        
#establish list of characters
charindex = set()
for w in vocab:
    for c in w:
        charindex.add(c)

## Empty crossword import

An empty crosswrod is a matrix of boolean values. False fields are black fields.

In [28]:
import numpy as np

In [61]:
# as a filler, we will use our example crossword and take the setting from it.

grid = ['moeras_bleken', 
        'o_sara_reep_a', 
        'kt_miljoen_cd', 
        'kat_atoom_tra',
        'arts_ons_klit', 
        '_w_tu_k_do_m_',
        'bereidvaardig',
        '_m_lt_r_sp_n_',
        'test_vos_step',
        'oer_meute_kei',
        'nl_uurwerk_lp',
        'i_snit_neon_e',
        'cruise_onecht']

emptygrid = np.full((len(grid), len(grid[0])), True, dtype=bool)
        
for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == '_':
            emptygrid[j,i] = False

## Class definitions

In [317]:
class Sequence:
    def __init__(self, length, cors):
        self.l = length
        self.cors = cors
        self.wordset = vocabdict[self.l]
        for i in range(self.l):
            self.letteroptions = self.UpdateLetters()
        
    def __len__(self):
        return self.l
    
    def UpdateLetters(self):
        optionslist = []
        for i in range(self.l):
            oldoptions = charindex
            options = set()
            for w in self.wordset:
                if len(w) > i:
                    char = w[i]
                    options.add(char)
                    if len(options) == len(oldoptions):
                        break
            optionslist.append(options)
        return optionslist
    
    def Letters(self, x, y):
        for i in range(len(self.cors)):
            xcor, ycor = self.cors[i]
            if (xcor, ycor) == (x,y):
                return self.letteroptions[i]   
            
    def ExcludeLetter(self, i, letter):
        newset = set()
        for w in self.wordset:
            if w[i] != letter:
                newset.add(w)
        self.wordset = newset
        self.UpdateLetters()
        
    def Choose(self, word):
        """Turn wordset into a single word. Return list of coordinates for which the letterset has changed."""
        self.wordset = {word}
        cors = list()
        for i in range(len(self)):
            if len(self.letteroptions[i]) > 1:
                self.letteroptions[i] = set(word[i])
                cors.append(self.cors[i])
        
        return cors
    
    def ExcludeWord(self, word):
        """Remove word from wordlist."""
        self.wordset.remove(word)
        newletteroptions = self.UpdateLetters()
        #this can be more efficient. You only need to see if one letter per position is still an option.
        to_update = list()
        for i in range(len(self)):
            if newletteroptions != self.letteroptions:
                to_update.append(self.cors[i])
        self.letteroptions = newletteroptions
        return to_update

In [336]:
class Crossword:
    def __init__(self, emptygrid):
        self.Empty = emptygrid
        shape = emptygrid.shape
        self.Width = shape[0]
        self.Height = shape[1]
        
        #array that maps a coordinate in the crossword to which sequences overlap with it
        self.ref = np.array([[(None,None) for x in range(self.Height)] for y in range(self.Width)], tuple)
        
        #add horizontal sequences
        self.hor = []
        for y in range(self.Height):
            left = False
            for x in range(self.Width - 1):
                current = self.Empty[x,y]
                right = self.Empty[x+1, y]
                if (left, current, right) == (False, True, True):
                    #if this is the start of a new sequence
                    #check length of sequence
                    l = 0
                    for value in self.Empty[x:, y]:
                        if value:
                            l += 1
                        else:
                            break
                               
                    cors = []
                    for xcor in range(x, x+l):
                        cors.append((xcor, y))
                        self.ref[xcor, y][0] = len(self.hor)
                    
                    self.hor.append(Sequence(l, cors))
                    
                left = current
        
        #add vertical sequences
        self.ver = []
        for x in range(self.Width): 
            above = False
            for y in range(self.Height - 1):
                current = self.Empty[x,y]
                below = self.Empty[x, y+1]
                if (above, current, below) == (False, True, True):
                    #if this is the start of a new sequence
                    #check length of sequence
                    l = 0
                    for value in self.Empty[x, y:]:
                        if value:
                            l += 1
                        else:
                            break
                          
                    cors = []
                    for ycor in range(y, y+l):
                        cors.append((x, ycor))
                        self.ref[x, ycor][1] = len(self.ver)
                    
                    self.ver.append(Sequence(l, cors))
                    
                above = current
        
    def SeqFromCors(self, x, y, direction):
        if direction == 'hor':
            if c.ref[x, y][0] != None:
                return c.hor[c.ref[cors[x], y][0]]
            return None
        else:
            if c.ref[x, y][1] != None:
                return c.ver[c.ref[x, y][1]]
            return None
    
    def Update(self, x, y, direction):
        """Update the wordset of a single sequence"""
        seq = self.SeqFromCors(x,y, direction)
        if seq:
            #set other direction
            if direction == 'hor':
                otherdir = 'ver'
            else:
                otherdir = 'hor'

            #loop through sequence's fields
            for i in range(len(seq.cors)):
                xcor, ycor = seq.cors[i]
                #get intersecting sequence
                otherseq = self.SeqFromCors(xcor, ycor, otherdir)

                if otherseq:
                    to_delete = seq.Letters(xcor, ycor) - otherseq.Letters(xcor, ycor)
                    if len(to_delete) > 0:
                        oldoptions = [o for o in seq.letteroptions]

                        for letter in to_delete:
                            seq.ExcludeLetter(i, letter)

                        #check if any other fields in the sequence have more narrow options now
                        narrowed_down = list()
                        for i in range(len(seq)):
                            if seq.cors[i] != (x,y):
                                if seq.letteroptions[i] != oldoptions[i]:
                                    narrowed_down.append(seq.cors[i][0], seq.cors[i][1], otherdir)

                        return narrowed_down


## Filling in the crossword

In [341]:
def update(queue, crossword):
    """update the crossword, given a queue"""
    if len(queue) > 0:
        #pop first sequence
        x,y,direction = queue[0]
        seq = crossword.SeqFromCors(x,y,direction)
        
        if seq:
            #update sequence's wordlist
            neighbourcors = crossword.Update(x,y,direction)

            #check if this led to a contradiction
            wordsleft = len(crossword.SeqFromCors(x,y,direction).wordset)
            if wordsleft > 0:
                if neighbourcors:
                    newqueue = queue[1:] + neighbourcors
                else:
                    newqueue = queue[1:]
                update(newqueue, crossword)
            else:
                return False
        else:
            update(queue[1:], crossword)
        
    else:
        return True

In [338]:
import random
import copy

def FillIn(x, y, direction, crossword):
    """Recursive function that fills in words in sequences"""
    
    sequence = crossword.SeqFromCors(x,y,direction)
    if direction == 'hor':
        otherdir = 'ver'
    else:
        otherdir = 'hor'
    solved = False
    
    while len(sequence.wordset) > 0 and solved == False:
        wordchoice = random.choice(list(sequence.wordset))
        
        newcw = copy.deepcopy(crossword)
        neighbours = [(x,y, otherdir) for x,y in newcw.SeqFromCors(x,y,direction).Choose(wordchoice)]
        updated = update(neighbours, newcw)
        
        if not updated: #meaning if updating led to a contradiction
            neighbours = [(x,y, otherdir) for x,y in sequence.ExcludeWord(wordchoice)]
            updated = update(neighbours, crossword)
        else:
            complete = True
            for sequence in crossword.hor + crossword.ver:
                if len(sequence.wordset) > 1:
                    complete = False
            
            if complete:
                solved = True
            else:
                newcors = crossword.NextSeq()
                following = FillIn(newcors)
                if not following:
                    neighbours = [(x,y, otherdir) for x,y in sequence.ExcludeWord(wordchoice)]
                    updated = update(neighbours, crossword)

    if len(sequence.wordset) == 0:
        return None
    if solved:
        return crossword

In [342]:
c =  Crossword(emptygrid)
s = c.SeqFromCors(0,0,'hor')
FillIn(0, 0, 'hor', c)

IndexError: tuple index out of range